In [7]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

Input Problem parameters

In [35]:
import json

l=["small", "medium", "large"]
res = dict()

for taille in l:
    nom_fichier = taille+".json"
    fichier = open(nom_fichier)
    print("I opened ", nom_fichier, " filed. \n")

    data = json.load(fichier)
    print("I loaded ", nom_fichier, " data in a dict. \n")

    res[taille] = data
    print("I stored ", nom_fichier, " data in res. \n")

    fichier.close()

I opened  small.json  filed. 

I loaded  small.json  data in a dict. 

I stored  small.json  data in res. 

I opened  medium.json  filed. 

I loaded  medium.json  data in a dict. 

I stored  medium.json  data in res. 

I opened  large.json  filed. 

I loaded  large.json  data in a dict. 

I stored  large.json  data in res. 



In [9]:
res["small"]

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}

In [4]:
res["medium"]

{'horizon': 22,
 'qualifications': ['F', 'G', 'A', 'D', 'J', 'C', 'H', 'E', 'B', 'I'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': [1, 2]},
  {'name': 'Liam', 'qualifications': ['A', 'D', 'E'], 'vacations': [1, 2]},
  {'name': 'Emma', 'qualifications': ['B', 'H'], 'vacations': [8, 9]},
  {'name': 'Noah',
   'qualifications': ['G', 'D', 'J', 'C', 'H', 'I'],
   'vacations': []},
  {'name': 'Amelia',
   'qualifications': ['F', 'G', 'J', 'E'],
   'vacations': [16, 15]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 20,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 16,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 12,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'C': 1}},
  {'name': 'Job4',
   'gain': 30,
   'due_date': 18,
   'daily_penal

In [5]:
res["large"]

{'horizon': 36,
 'qualifications': ['E', 'I', 'J', 'A', 'C', 'H', 'G', 'D', 'B', 'F'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['B', 'A', 'C'],
   'vacations': [1, 2, 3, 4, 5]},
  {'name': 'Liam',
   'qualifications': ['E', 'A', 'D'],
   'vacations': [6, 7, 8, 9, 10]},
  {'name': 'Emma',
   'qualifications': ['B', 'H'],
   'vacations': [21, 22, 23, 24, 25]},
  {'name': 'Noah',
   'qualifications': ['C', 'H', 'G', 'D'],
   'vacations': [26, 27, 28, 29, 30]},
  {'name': 'Amelia',
   'qualifications': ['E', 'I', 'G', 'J', 'F'],
   'vacations': []},
  {'name': 'Oliver',
   'qualifications': ['J', 'F', 'G'],
   'vacations': [32, 33, 31]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 26,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 22,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 16,
   'd

In [6]:
l

['small', 'medium', 'large']

In [36]:
## à changer selon le fichier que l'on veut
## traiter
taille = "small"

In [37]:
horizon = res[taille]["horizon"]
qualifications = res[taille]["qualifications"]
staff = res[taille]["staff"]
jobs = res[taille]["jobs"]

In [38]:
noms_employes = []
for person in staff:
    noms_employes.append(person["name"])

noms_projet = []
for projet in jobs:
    noms_projet.append(projet["name"])

In [39]:
nombre_projets = len(noms_projet)
nombre_qualif = len(qualifications)
nombre_employes = len(noms_employes)

Définissons les variables

In [77]:
## besoin_i_j, le projet j a besoin de la qualif i
besoin = [[0]*nombre_projets]*nombre_qualif

## conge_k_n, l'employé k est en congés au jours n
conge = [[0]*horizon]*nombre_employes

## qualif_i_k, l'employé k a la qualif i
qualif = [[0]*nombre_employes]*nombre_qualif

## d_j, date de livraison selon les projets
d = [0]*nombre_projets

## p_j, pénalités selon les projets
p = [0]*nombre_projets

## b_j, bénéfices selon les projets
b = [0]*nombre_projets




##########################################
##########################################



for job in jobs:
    ## jobs is list of dict
    ## job is dict

    num_projet = noms_projet.index(job["name"])
    b[num_projet] = job["gain"]
    d[num_projet] = job["due_date"]
    p[num_projet] = job["daily_penalty"]

    for qualification, jours in job["working_days_per_qualification"].items():
        num_qualif = qualifications.index(qualification)
        besoin[num_qualif][num_projet] = jours
        print(besoin)

# print("besoin : ", besoin)
# print("qualif : ", qualif)
# print("conge : ", conge)
# print("d : ", d)
# print("p : ", p)
# print("b : ", b)

[[1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]]
[[1, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]]
[[1, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 0]]
[[1, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 0]]
[[1, 2, 1, 0, 0], [1, 2, 1, 0, 0], [1, 2, 1, 0, 0]]
[[1, 2, 1, 2, 0], [1, 2, 1, 2, 0], [1, 2, 1, 2, 0]]
[[1, 2, 1, 2, 0], [1, 2, 1, 2, 0], [1, 2, 1, 2, 0]]
[[1, 2, 2, 2, 0], [1, 2, 2, 2, 0], [1, 2, 2, 2, 0]]
[[1, 2, 2, 1, 0], [1, 2, 2, 1, 0], [1, 2, 2, 1, 0]]
[[1, 2, 2, 1, 2], [1, 2, 2, 1, 2], [1, 2, 2, 1, 2]]


In [ ]:
for person in staff:
    num_person = noms_employes.index(person["name"])

    for qual in person["qualifications"]:    
        num_qualif = qualifications.index(qual)
        qualif[num_person][num_qualif] = 1
    
    for date in person["vacations"]:
        if date is not None:
            conge[num_person][date] = 1

# print("besoin : ", besoin)
# print("qualif : ", qualif)
# print("conge : ", conge)
# print("d : ", d)
# print("p : ", p)
# print("b : ", b)

Définissons les paramètres

In [ ]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon]*nombre_employes]*nombre_projets]*nombre_qualif